# TP 1 - Aprendizaje de Maquina

Alumno: Rodrigo Pazos

## Consigna

Una plataforma de ventas online nos contrata para que realicemos un modelo que nos
permita detectar un posible fraude dada cierta operación para ello contamos con un dataset
que contiene las siguientes columnas:

- Step: representa una unidad de tiempo donde 1 step equivale a 1 hora
- type: tipo de transacción en línea
- amount: el importe de la transacción
- nameOrig: cliente que inicia la transacción
- oldbalanceOrg: saldo antes de la transacción
- newbalanceOrig: saldo después de la transacción
- nameDest: destinatario de la transacción
- oldbalanceDest: saldo inicial del destinatario antes de la transacción
- newbalanceDest: el nuevo saldo del destinatario después de la transacción
- isFraud: transacción fraudulenta

Utilizando los modelos de clasificación vistos hasta el momento generar un notebook que
permita de ser posible resolver el problema que nos está planteando el cliente.
IMPORTANTE
Sabemos que por cada transacción aprobada el porcentaje de ganancia es de un
20%, y por cada fraude aprobado se pierde el 100% del dinero de la transacción.
Realizar un análisis y determinar un modelo que permita maximizar la ganancia de la
empresa.

## Solucion

In [2]:
import pandas as pd
import numpy as np

### Carga y preparacion de datos

In [3]:
df = pd.read_csv("./data/PS_20174392719_1491204439457_log.csv")
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


Usamos get dummies para encodear la columna type

In [4]:
df = pd.get_dummies(df, columns=["type"])
df.head(5)

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0,0,0,1,0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0,0,0,1,0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0,0,0,0,1
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,0,1,0,0,0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,0,0,0,1,0


In [5]:
len(df["nameOrig"].unique())

6353307

No se puede usar get dummies para name porque hay muchas cuentas diferentes. Ademas como hay casi la misma cantidad de rows como de cuentas de origen diferentes podriamos considerar que no suma mucha informacion o que si lo hay es marginal

In [6]:
len(df["nameDest"].unique())

2722362

Aunque mejora un poco porque hay menos cuentas unicas de destino sigue sin ser algo manejable o practico para este dataset. Entonces se determina eliminar estas dos columnas ya que no suman ningun valor

In [7]:
x_df = df.drop(columns=["nameDest", "nameOrig", "isFraud"])
y_df = df[["isFraud"]]

In [8]:
x_df.head(5)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,0,1,0


In [9]:
y_df.head(5)

,isFraud
0,0
1,0
2,1
3,1
4,0


In [10]:
X = x_df.values
y = y_df.values

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5090096, 12)
(1272524, 12)
(5090096, 1)
(1272524, 1)


## Modelos y entrenamiento

Antes de entrenar es necesario un contexto teorico general del analisis que se hara mas adelante 

Recall, también conocido como tasa de verdaderos positivos o sensibilidad, mide la capacidad de un modelo para identificar correctamente las instancias positivas dentro del total de instancias positivas reales en el conjunto de datos

La detección de fraudes generalmente enfrenta un problema de desequilibrio de clases, donde el número de transacciones no fraudulentas supera con creces el número de transacciones fraudulentas. En estos casos, un modelo ingenuo que predice todas las instancias como no fraudulentas podría lograr una alta precisión debido al gran número de verdaderos negativos. Sin embargo, este enfoque no lograría identificar la mayoría de las transacciones fraudulentas (alta tasa de FN), lo cual es inaceptable en la detección de fraudes.

El recall se calcula como la proporción de verdaderos positivos respecto a la suma de verdaderos positivos y falsos negativos:

Recall = VP / (VP + FN)

En la detección de fraudes, un alto recall indica que el modelo es efectivo para identificar una parte significativa de las transacciones fraudulentas. Un alto recall implica una menor tasa de FN, lo que significa menos casos donde el fraude real pasa desapercibido.

Es decir, que mas alla de que se consideraran todas las metricas la metrica mas relevante para este proceso es la de recall. Esta metrica nos a permitir entender mejor la potencia de cada uno de los modelos

#### Ganancia

La consigna aclara ademas como afecta a las ganancias la capacidad predictiva del modelo, programada en el siguiente bloque de codigo

In [49]:
amount = X_test[:, 1]

def calculate_revenue(amount, y_true, y_pred):
    zipped = np.stack((y_true, y_pred), axis=1)
    accepted_frauds = np.all(zipped == [1, 0], axis=1)
    accepted_legit = np.all(zipped == [0, 0], axis=1)
    revenue = amount * accepted_legit.astype(int) * 0.2
    loss = amount * accepted_frauds.astype(int)
    return (revenue-loss).sum()

### Naive classifaction

El approach mas sencillo para la empresa seria el de asumir que todas las transacciones no son fraudulentas

In [55]:
from sklearn.metrics import classification_report

print(y_test.shape)

naive_prediction = np.zeros(y_test.shape)

print(classification_report(y_true=y_test, y_pred=naive_prediction))

naive_revenue = calculate_revenue(amount, y_test.flatten(), naive_prediction.flatten())

print(f"La ganancia usando un modelo basico sin ninguna inteligencia es {naive_revenue:.3}")

(1272524, 1)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270883
           1       0.00      0.00      0.00      1641

    accuracy                           1.00   1272524
   macro avg       0.50      0.50      0.50   1272524
weighted avg       1.00      1.00      1.00   1272524

La ganancia usando un modelo basico sin ninguna inteligencia es 4.28e+10


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La ganancia usando un modelo basico sin ninguna inteligencia es 4.28e+10. Esto se puede usar de linea de base para comprar la performance de los modelos a entrenar

### Logistic Regression

Este primer approach es un entrenamiento generico, sin estudiar ningun hiperparametro

In [22]:
log_reg = LogisticRegression()

log_reg.fit(X_train, y_train.ravel())

LogisticRegression()

In [13]:
y_pred_log_reg = log_reg.predict(X_test)
y_pred_log_reg.shape

(1272524,)

In [20]:
print(classification_report(y_true=y_test, y_pred=y_pred_log_reg))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270883
           1       0.35      0.41      0.38      1641

    accuracy                           1.00   1272524
   macro avg       0.67      0.71      0.69   1272524
weighted avg       1.00      1.00      1.00   1272524



### Decision tree

Este primer approach es un entrenamiento generico, sin estudiar ningun hiperparametro

In [33]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [34]:
y_pred_dt = dt.predict(X_test)
y_pred_dt.shape

(1272524,)

In [35]:
print(classification_report(y_true=y_test, y_pred=y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270883
           1       0.89      0.88      0.89      1641

    accuracy                           1.00   1272524
   macro avg       0.95      0.94      0.94   1272524
weighted avg       1.00      1.00      1.00   1272524



A partir de esto se puede determinar que en este primer entrenamiento generico Decission Tree da muy buenos resultados, con un recall de 0.94, que es considerablemente mejor al recall de Logistic Regression.

A partir de esta funcion calculamos la ganancia con cada uno de los modelos

In [63]:
revenue_dt = calculate_revenue(amount, y_test.flatten(), y_pred_dt)
revenue_log_reg = calculate_revenue(amount, y_test.flatten(), y_pred_log_reg)

print(f"La ganancia usando un modelo basico de DT es {revenue_dt:.3}. Mejora un {((revenue_dt/naive_revenue)-1)*100:.3f}% sobre la linea de base")
print(f"La ganancia usando un modelo basico de Log Reg es {revenue_log_reg:.3}. Mejora un {((revenue_log_reg/naive_revenue)-1)*100:.3f}% sobre la linea de base")

print(f"Conviene usar esta version de DT por una diferencia de {revenue_dt - revenue_log_reg:.3}")

La ganancia usando un modelo basico de DT es 4.51e+10. Mejora un 5.549% sobre la linea de base
La ganancia usando un modelo basico de Log Reg es 4.47e+10. Mejora un 4.531% sobre la linea de base
Conviene usar esta version de DT por una diferencia de 4.36e+08


## Conclusion

La motivacion para seguir mejorando el modelo no es tan grande: ambos sistemas fueron relativamente costosos de entrenar y seguramente son mas costosos de estudiar en profunidad para ajustar los hiperparametros. Dado que la mejora de usar un modelo con recall mucho mejor (0.94 > 0.5) fue relativamente marginal (5% de ganancia extra) mejorar ese recall va a ser costoso y probablemente la mejora en el resultado de la ganancia no lo amerite. Esto se nota mejor si vemos la diferencia en metricas entre el DT y la regresion logistica: todas las metricas de DT son considerablemente mejores y la diferencia en la ganancia de un modelo y otro es poco mas de un 1%